# Часть 2: Создание MCP-сервера

В этом примере мы создадим собственный MCP-сервер с использованием FastMCP и подключим его к агенту через STDIO транспорт.

MCP решает проблемы обычного tool calling через стандартизацию и переиспользуемость.

## Как это работает?

MCP - протокол - по сути из себя представляет HTTP ручки с определенным порядком вызова, в котором обмен построен на стандартизированных JSON-сообщениях.

Задействуется три компонента:

1. **MCP Server** - отдельная программа (локальный процесс через stdio или удалённый сервис через HTTP+SSE), которая предоставляет доступ к ресурсам, **инструментам** и промптам через стандартизированный MCP-интерфейс на базе JSON-RPC 2.0. Например, серверы для работы с файловой системой, историей коммитов, базами данных и т.д.
2. **MCP Client** - модуль внутри хоста, который устанавливает соединение с конкретным MCP-сервером, обменивается JSON-RPC сообщениями и управляет жизненным циклом соединения.
3. **Host** - приложение-контейнер (IDE-ассистент или ваш Python-процесс с мультиагентной системой), которое содержит LLM, управляет MCP-клиентами, координирует взаимодействие с пользователем и агрегирует контекст от разных серверов.

Основная схема такая: Знакомство >> Запрос >> Ответ

- На **первом** шаге мсп-клиент спрашивает у мсп-сервера что второй умеет. В ответ сервер показывает свое описание и список доступных инструментов. Выглядеть это может так:

```json
// Сообщение от MCP Server -> MCP Client
{
  "jsonrpc": "2.0",
  "result": {
    "tools": [{
        "name": "calculator_arithmetic",  // уникальное имя операции
        "title": "Calculator",
        "description": "Perform mathematical calculations including basic arithmetic, trigonometric functions, and algebraic operations",  // понятное для LLM описание
        "inputSchema": {  // параметры метода (тула)
          "type": "object",
          "properties": {
            "expression": {
              "type": "string",
              "description": "Mathematical expression to evaluate (e.g., '2 + 3 * 4', 'sin(30)', 'sqrt(16)')"
            }
          },
          "required": ["expression"]
        }
      }
  ]}
}
```

    Затем хост передает такого вида сообщение вместе с воепросом в LLM. Затем сама модель решает что делать и знает, что у нее есть инструмент (в данном примере - калькулятор).

- На **втором** шаге LLM анализирует вопрос и решает нужно ли для ответа вызвать тул. НАпример с `experession = "2 + 2 * 2"`. Хост через клиент отправляет запрос на сервер такого вида:

```json
// Сообщение от Host через MCP Client -> MCP Server
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "calculator_arithmetic",
    "arguments": { "expression": "2 + 2 * 2" }
  }
}
```
    Это и есть вызов функции-тула

- На **третьем** шаге сервер получает запрос, выполняет вычисления и возращает результат клиенту, а тот передает хосту, который затем направляет в LLM. Пример результата:

```json
// Сообщение от MCP Server -> MCP Client -> Host
{
  "jsonrpc": "2.0",
  "id": 3,
  "result": {
    "content": [{
        "type": "text",
        "text": "8"
    }]
  }
}
```
    Теперь модель решает готова ли она дать окончательный ответ.

> **Важно**: информация о подключенных мсп-серверах и мета-информация из них съедают контекст LLM. Нужно быть аккуратнее с описанием серверов и тулов. Также, не стоит расчитывать, что дав модели 100 мсп-серов вы будете получать хороший результат, скорее вы просто запутаете модель или заполните ей контекстое окно.

## Какие использовать библиотеки?

В данном примере рассмотрим [FastMCP](https://gofastmcp.com/getting-started/welcome) (обертка над Starlette), тк эта библиотека активно развивается и поддерживает много фич. Но главное здесь - соблюдать протокол. Поэтому можно использовать и другие фреймворки, включая [оффициальную имплементацию MCP](https://github.com/modelcontextprotocol/python-sdk). **В том числе на других языках.**

## Шаг 1: Импорты и настройка

In [1]:
import asyncio
import os
from pathlib import Path

from dotenv import load_dotenv
from fastmcp import Client
from langchain.agents import create_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_openai import ChatOpenAI

load_dotenv()

BASE_MODEL = os.getenv("BASE_MODEL") or "qwen/qwen3-235b-a22b-2507"

<frozen abc>:106: LangGraphDeprecatedSinceV10: AgentStatePydantic has been moved to `langchain.agents`. Please update your import to `from langchain.agents import AgentStatePydantic`. Deprecated in LangGraph V1.0 to be removed in V2.0.


## Шаг 2: Структура MCP-сервера

Наш сервер находится в файле `server_stdio.py`. Он использует FastMCP и предоставляет два инструмента:

- `convert_document` - конвертирует документы в markdown
- `analyze_document` - возвращает статистику о документе

Сервер запускается как отдельный процесс и общается через STDIO транспорт.

## Шаг 3: Подключение к серверу

Создаем клиент, который подключается к нашему серверу через STDIO.
Сервер автоматически предоставляет список своих инструментов через `tools/list`.

In [2]:
async def main():
    server_script = Path("server_stdio.py")

    async with Client(str(server_script)) as client:
        print("Available tools:")
        tools_list = await client.list_tools()
        for tool in tools_list:
            print(f"  - {tool.name}")

        tools = await load_mcp_tools(client.session)

        llm = ChatOpenAI(model=BASE_MODEL, temperature=0)

        agent = create_agent(llm, tools)

        test_pdf = Path("../../data/test.pdf")

        response = await agent.ainvoke(
            {
                "messages": [
                    (
                        "user",
                        f"Analyze the document structure at {test_pdf} and create laconic summary of the text.",
                    )
                ]
            }
        )

        print("Agent response:")
        for message in response["messages"]:
            if (
                message.type == "ai"
                and hasattr(message, "content")
                and message.content
            ):
                print(f"\n{message.content}")

## Шаг 4: Запуск примера

Выполним асинхронную функцию main для взаимодействия с сервером.

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
asyncio.run(main())

Available tools:
  - convert_document
  - analyze_document
Agent response:

The manul (Otocolobus manul) is a small wild cat species adapted to cold, arid steppe and semi-desert ecosystems across Central and Western Asia. It exhibits unique morphological and behavioral traits, including dense fur for insulation, cryptic coloration for camouflage, and reliance on rocky outcrops and burrows for shelter. The species is solitary, with males having larger home ranges (avg. 98.8 km²) than females (avg. 23.1 km²) in Mongolia, and shows seasonal breeding patterns tied to photoperiod.

Manuls primarily feed on small mammals, especially pikas, which constitute over 50% of their diet. They employ various hunting techniques such as stalking, ambush, and flushing prey. Despite a broad geographic range, populations are sparse and patchily distributed due to habitat specialization, predation pressure, and low prey availability.

Threats include predation by raptors, foxes, wolves, and domestic dogs, 